In [11]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [6]:
sc = SparkContext("local", "testing")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/29 09:57:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
lst = list(map(int, input("Enter space separated list of integers: ").split()))
lst

[1, 2, 3, 4, 5]

In [9]:
sc.parallelize(lst).map(lambda x: x**2).collect()

[1, 4, 9, 16, 25]

In [16]:
spark = SparkSession.builder.appName("SparkOperations").getOrCreate()

df_lst = []
for i in lst:
    df_lst.append([i])
df = spark.createDataFrame(df_lst, ["num"])
df.show()

+---+
|num|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



In [17]:
df.agg({'num': 'max'}).show()

+--------+
|max(num)|
+--------+
|       5|
+--------+



In [18]:
df.agg({'num': 'avg'}).show()

+--------+
|avg(num)|
+--------+
|     3.0|
+--------+



In [58]:
import pandas as pd
data = pd.read_csv('data.csv')
data


,id,first_name,last_name,a,b
0,1,sravan,vignan,67,89
1,2,ojaswi,vvit,78,89
2,3,rohith,vvit,100,80
3,4,sridevi,vignan,78,80
4,5,sravan,vignan,89,98
5,6,gnanesh,iit,94,98


In [59]:
df = spark.read.csv('data.csv', header=True, escape="\"")
df.show(3)

+---+----------+---------+---+---+
| id|first_name|last_name|  a|  b|
+---+----------+---------+---+---+
|  1|    sravan|   vignan| 67|89 |
|  2|    ojaswi|     vvit| 78|89 |
|  3|    rohith|     vvit|100|80 |
+---+----------+---------+---+---+
only showing top 3 rows



In [60]:
df.schema

StructType([StructField('id', StringType(), True), StructField('first_name', StringType(), True), StructField('last_name', StringType(), True), StructField('a', StringType(), True), StructField('b', StringType(), True)])

In [65]:
print("No. of rows:", df.count())
df.agg({"a": "max"}).show()
df.agg({"a": "min"}).show()
df.agg({"a": "std"}).show()
df.agg({"a": "mode"}).show()
df.agg({"a": "median"}).show()

No. of rows: 6
+------+
|max(a)|
+------+
|    94|
+------+

+------+
|min(a)|
+------+
|   100|
+------+

+----------------+
|       stddev(a)|
+----------------+
|12.1764800606196|
+----------------+

+-------+
|mode(a)|
+-------+
|     78|
+-------+

+---------+
|median(a)|
+---------+
|     83.5|
+---------+

